In [1]:
import itk
import pydicom as dcm
import matplotlib.pyplot as plt
import numpy as np
from itkwidgets import view
from skimage import io
filename = "RD1.2.752.243.1.1.20191212115003150.1740.20852.dcm"
ds = dcm.dcmread(filename)
print("Dicom data:")
print(ds)

Dicom data:
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0012) Instance Creation Date              DA: '20171214'
(0008, 0013) Instance Creation Time              TM: '130459'
(0008, 0016) SOP Class UID                       UI: RT Dose Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.752.243.1.1.20191212115003150.1740.20852
(0008, 0020) Study Date                          DA: '20171211'
(0008, 0030) Study Time                          TM: '174343'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'RTDOSE'
(0008, 0070) Manufacturer                        LO: 'RaySearch Laboratories'
(0008, 0090) Referring Physician's Name          PN: 'Anonymized'
(0008, 1030) Study Description                   LO: 'Anonymized'
(0008, 103e) Series Description                  LO: 'Anonymized'
(0008, 1070) Operators' Name                     PN: ''
(0008, 1090) Manufacturer's Model Name           LO: 

In [2]:
#use itkwidgets to visualize and play with the user interface
view(ds.pixel_array)

#Raising gradient opacity makes it easier to see deeper into the brain
#However, when gradient opacity is too high, all structure is lost.

#X, Y, and Z planes can be inserted and moved. This allows doctors to 
#analyze the brain layer by layer.

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUS3; proxy …

In [3]:
#slicing the pixel_array in half gives an image of the bottom half of the image stack
bhalf = ds.pixel_array[:len(ds.pixel_array)//2:]
view(bhalf)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUS3; proxy …

In [4]:
#Drawing planes in the CT scan can better allow doctors to notice assymetries in a patient's brain
#By changing the roi boundaries, a doctor can narrow down the image to important areas of the brain
#I would like to figure out how to programmtically set roi boundaries
view(ds.pixel_array, slicing_planes=True, select_roi=True)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUS3; proxy …

In [5]:
#read all CT slices (2D) using skimage (function io.imagecollection) and visualize
slices = io.ImageCollection('../slices/*.png')
print('Type:', type(slices))
#the ImageCollection length is 0 but I'm not sure why.
print('Length:', len(slices))

Type: <class 'skimage.io.collection.ImageCollection'>
Length: 0


In [6]:
#perform matrix multiplication and compute Hadarmard product

#init_mat initializes an array with r rows and c cols
#The first entry has value 1 and each consecutive entry is incremented by 1
def init_mat(r, c):
    mat = np.empty(shape=(r,c))
    for i in range(r):
        for k in range(c):
            mat[i][k] = i * c + k + 1
    return mat

#matrix_mat performs matrix multiplication on 2D matrices
def matrix_mat(A, B):
    if A.shape[1] != B.shape[0]:
        raise ValueError("Mismatched dimensions") 
    dim = A.shape[1] 
    r = A.shape[0]
    c = B.shape[1]
    mat = np.empty(shape=(r,c))
    for i in range(r):
        for k in range(c):
            sum = 0
            for j in range(dim):
                sum += A[i][j] * B[j][k]
            mat[i][k] = sum
    return mat

#hadarmard_product computes the Hadarmard product of two 2D matrices
def hadarmard_product(A, B):
    if A.shape[0] != B.shape[0] or A.shape[1] != B.shape[1]:
        raise ValueError("Mismatched dimensions")
    mat = np.empty(shape = A.shape)
    for i in range(mat.shape[0]):
        for k in range(mat.shape[1]):
            mat[i][k] = A[i][k] * B[i][k]
    return mat
            
A = init_mat(4, 5)
print("Matrix A:")
print(A)
B = init_mat(5, 3)
print("Matrix B:")
print(B)
print("Matrix product of A and B:")
print(matrix_mat(A,B)) #equivalent to calling A.dot(B)

#Hadarmard product can only be computed for 2 matrices with the same dimensions
C = init_mat(4, 4)
print("Matrix C:")
print(C)
print("Hadardmard product of C and itself:")
print(hadarmard_product(C, C))

Matrix A:
[[ 1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10.]
 [11. 12. 13. 14. 15.]
 [16. 17. 18. 19. 20.]]
Matrix B:
[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]
 [10. 11. 12.]
 [13. 14. 15.]]
Matrix product of A and B:
[[135. 150. 165.]
 [310. 350. 390.]
 [485. 550. 615.]
 [660. 750. 840.]]
Matrix C:
[[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]
 [13. 14. 15. 16.]]
Hadardmard product of C and itself:
[[  1.   4.   9.  16.]
 [ 25.  36.  49.  64.]
 [ 81. 100. 121. 144.]
 [169. 196. 225. 256.]]
